# WALL-E's Lost Memories

One ordinary Earth-cleaning afternoon, WALL·E climbed a pile of old iPhones trying to rescue a Rubik’s Cube. A pigeon startled him. He slipped.

**CRASH.**

*Oh no... The fall seemed pretty bad. Let's try asking him a couple of questions:*

> 🧑‍🔧: It's okay, buddy. You took a pretty bad fall. Let’s try something simple. Who are you?  
🤖: I... I do not know. Memory blocks missing.  
🧑‍🔧: Hmm. Okay. Let’s try this... Do you remember EVE?  
🤖: E...V...E... error. No match found in memory banks. Who... is EVE?  

Oh my. The fall WAS pretty bad. Seems like our little robot lost all his memory.  

But wait! We found an ancient relic in a dusty old USB: the movie script of WALL·E!

In [ ]:
# Obtain the script! We are nice enough to locate and prepare it for you
!curl -L "https://assets.scriptslug.com/live/pdf/scripts/wall-e-2008.pdf?v=1729115058" -o walle_script.pdf

We can use it to rebuild his memories using **Retrieval Augmented Generation** powered by **LangChain**.

## What is Retrieval Augmented Generation (RAG)?

## 🚀 Step 0: Fire Up WALL·E’s Core Systems (Environment Setup)

In [ ]:
# Install the main langchain package
!pip install langchain

# Install the langchain dependencies required for our project
!pip install --quiet --upgrade langchain-core langchain-text-splitters langchain-community langgraph langchain-openai

# Install other dependencies
%pip install --quiet --upgrade pypdf sentence_transformers

In [ ]:
# Set up OpenAI API key
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [ ]:
# Set up embeddings model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# Set up vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
# Set up chat model
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## 📜 Step 1: Load the Memory Archive (Document Loader)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF script into WALL·E's recovery core
loader = PyPDFLoader("walle_script.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
print(f"Total pages loaded: {len(pages)}\n")

# Skipping the title page (pages[0])
page_num = 1
print(f"{'='*40}")
print(f"📄 Page {page_num} Metadata")
print(f"{'-'*40}")
print(pages[page_num].metadata)

print(f"\n{'='*40}")
print(f"📜 Page {page_num} Content")
print(f"{'-'*40}")
print(pages[page_num].page_content)
print(f"{'='*40}\n")

Looks neat!

Let’s try feeding WALL·E the entire script. See if it works.  

We gently place the entire WALL·E script PDF into his input slot.  

**Beep... Whirr... BZZZT...**  

Uh-oh...  

> “Eeee–...ERR–...💥”

Looks like our sweet trash-compactor can’t process that much information at once. His tiny memory unit just isn’t built for entire movie scripts in one go.

More technical explanation here

To help WALL·E actually remember things, we need to split the script into smaller, memory-safe chunks.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(pages)

print(f"Split script into {len(all_splits)} sub-documents.")

## 🧲 STEP 3: Upload to WALL·E's Memory Module (Vector Store)

WALL·E doesn’t remember like humans. He needs vector memory!

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

In [ ]:
print(document_ids[:3])

Now we can search his memory by converting questions into embeddings and looking for the closest matches.

## 🧠 Step 4: Reconstruct Thoughts – Create the RAG Chain

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
question = "Who is Eve?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)

In [ ]:
print(answer)

## 🤖 Step 5: Ask WALL·E Questions About His Past